In [1]:
import numpy as np
import onnxruntime as ort
import cv2
import mlflow
from mlflow.tracking import MlflowClient

In [2]:
MLFLOW_URI = "http://localhost:8080"
MODEL_NAME = "anatomy_detector"
ALIAS = "champion"
TARGET_HEIGHT = 400
TARGET_WIDTH = 600

In [ ]:
mlflow.set_tracking_uri(MLFLOW_URI)
client = MlflowClient()
model_version = client.get_model_version_by_alias(name=MODEL_NAME, alias=ALIAS)
onnx_source = model_version.source
ONNX_MODEL_PATH = mlflow.artifacts.download_artifacts(onnx_source)

In [ ]:
# Load ONNX model
ort_session = ort.InferenceSession(ONNX_MODEL_PATH)

# test with image 1528.png 
test_img_path = r"dataset/Dataset for Fetus Framework/Internal Test Set/Standard/1528.png"

# Preprocess the image
test_img = cv2.imread(test_img_path, cv2.IMREAD_GRAYSCALE)      # Load as grayscale
test_img = cv2.resize(test_img, (TARGET_WIDTH, TARGET_HEIGHT))  # Resize to target dimensions
test_img = ((test_img / 255.0 - 0.5) / 0.5).astype(np.float32)                # Normalize to [0, 1]
test_img = np.expand_dims(test_img, axis=0)                     # Add batch dimension
test_img = np.expand_dims(test_img, axis=0)                     # Add channel dimension

# Run inference
ort_inputs = {'input': test_img}
ort_outputs = ort_session.run(None, ort_inputs)
pred_class_conf = np.array(ort_outputs[0].squeeze(0))   # [K]
pred_boxes = np.array(ort_outputs[1].squeeze(0))        # [K, 4]

# Print results
print(f"Predicted class confidences: {pred_class_conf}")
print(f"Predicted boxes: {pred_boxes}")